In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy import stats
import warnings
import fitz
import logging
import traceback
warnings.filterwarnings('ignore')

class IPASectorMapper:
    """
    Classe para mapeamento e agregação de setores IPA
    """
    def __init__(self):
        # Mapeamento explícito para garantir consistência na agregação
        self.explicit_mapping = {
            ## AGROPECUÁRIA
            # Agriculta e pecuária
            '201101000': '2011010', # Agricultura
            '201101500': '2011010', # Pecuária
            ## INDÚSTRIA EXTRATIVA
            # Indústria extrativa, inclusive petróleo e gás natural
            '202106000': '2021060',  # Extração de Petróleo e gás natural
            '202107000': '2021060',  # Extração de minerais metálicos e não-metálicos
            '202107200': '2021060',  # Minerais metálicos e não-metálicos
            '202108000': '2021060',  # Minerais metálicos e não-metálicos
            ## INDÚSTRIA DE TRANSFORMAÇÃO
            # Alimentos e bebidas
            '202210100': '2022100',  # Carne e derivados
            '202210200': '2022100',  # Pescados
            '202210300': '2022100',  # Conservas de frutas, legumes e outros vegetais
            '202210400': '2022100',  # Óleos e gorduras vegetais
            '202210500': '2022100',  # Laticínios
            '202210600': '2022100',  # Produtos amiláceos e alimentos para animais
            '202210700': '2022100',  # Açúcar
            '202210800': '2022100',  # Café
            '202210900': '2022100',  # Outros produtos alimentícios
            '202211000': '2022100',  #Bebidas
            # Fumo processado e produtos do fumo
            # Têxteis, vestuário, couros e calçados
            '202213000': '2022130',  # Produtos Têxteis (fios, tecitos e artefatos têxteis)
            '202214000': '2022130',  # Artigos do vestuário
            '202215000': '2022130',  # Couro e calçados
            # Madeira, celulose e papel
            '202216000': '2022160',  # Madeira desdobrada e produtos de madeira
            '202217100': '2022160',  # Celulose
            '202217200': '2022160',  # Papel, cartolina e papel-cartão
            '202217300': '2022160',  # Embalagens de papel, papelão ondulado, cartolina e papel-cartão
            '202217900': '2022160',  # Indústria de celulose e produtos de papel e papelão
            '202217400': '2022160',  # Artefatos diversos de papel, cartolina e papel-cartão
            # Produtos derivados do petróleo e biocombustíveis
            '202219200': '2022190',  # Produtos derivados do petróleo
            '202219300': '2022190',  # Biocombustíveis
            '202219900': '2022190',  # Produtos derivados do petróleo e Biocombustíveis
            # Indústria química, exceto petroquímica
            '202220100': '2022200',  # Produtos químicos inorgânicos
            '202220200': '2022200',  # Produtos químicos orgânicos
            '202220300': '2022200',  # Resinas e elastômeros
            '202220400': '2022200',  # Fibras e filamentos sintéticos
            '202220500': '2022200',  # Defensivos agrícolas e desinfestantes domissanitários
            '202220600': '2022200',  # Produtos de limpeza, cosméticos e artigos de perfumaria e de higiene pessoal
            '202220700': '2022200',  # Tintas, vernizes, esmaltes, lacas e produtos afins
            '202220900': '2022200',  # Produtos e preparados químicos diversos
            # Produtos farmacêuticos
            '202221200': '2022210',  # Medicamentos
            # Artigos de borracha e de material plástico
            '202222100': '2022220',  # Artigos de borracha
            '202222200': '2022220',  # Produtos de material plástico
            # # Metalurgia e siderurgia e produtos de minerais não-metálicos
            '202223100': '2022240',  # Vidro e produtos de vidro
            '202223200': '2022240',  # Cimento
            '202223300': '2022240',  # Artefatos de concreto, cimento e fibrocimento
            '202223400': '2022240',  # Produtos cerâmicos
            '202223900': '2022240',  # Pedras, cal e outros produtos não-metálicos
            '202224100': '2022240',  # Ferro-gusa e ferro-ligas
            '202224200': '2022240',  # Produtos siderúrgicos
            '202224400': '2022240',  # Produtos da metalurgia dos não-ferrosos
            '202225300': '2022240',  # Forjados e estampados
            '202225900': '2022240',  # Produtos diversos de metal 
            # Máquinas, equipamentos e aparelhos diversos
            '202226200': '2022270',  # Computadores e Acessórios
            '202226300': '2022270',  # Equipamentos transmissores de comunicação e aparelhos telefônicos
            '202227100': '2022270',  # Geradores, transformadores e motores elétricos
            '202227200': '2022270',  # Pilhas, baterias e acumuladores
            '202227300': '2022270',  # Equipamentos para distribuição e controle de energia elétrica
            '202227500': '2022270',  # Eletrodomésticos
            '202227900': '2022270',  # Equipamentos e aparelhos elétricos diversos
            '202228100': '2022270',  # Motores, bombas, compressores e equipamentos de transmissão
            '202228200': '2022270',  # Máquinas e equipamentos de uso geral
            '202228300': '2022270',  # Máquinas e equipamentos para a agricultura
            '202228400': '2022270',  # Máquinas-ferramenta
            '202228500': '2022270',  # Máquinas e equipamentos de terraplenagem e pavimentação
            '202228600': '2022270',  # Máquinas e equipamentos de uso específico
            # Veículos automotores e e outros equipamentos de transporte
            '202229100': '2022290',  # Automóveis, camionetas e utilitários
            '202229200': '2022290',  # Caminhões e ônibus
            '202229300': '2022290',  # Cabines, carrocerias e reboques
            '202229400': '2022290',  # Peças e acessórios para veículos automotores
            '202230100': '2022290',  # Embarcações
            '202230300': '2022290',  # Veículos ferroviários
            '202230900': '2022290',  # Outros equipamentos de transporte
            # Móveis
            '202231100': '2022310'  # Moveis 
        }
        
        # Nomes dos setores de 7 dígitos
        self.sector_names = {
            '2011010': 'Agricultura e pecuária', 
            '2021060': 'Indústria extrativa, inclusive petróleo e gás natural',
            '2022100': 'Alimentos e bebidas',
            '2022130': 'Têxteis, vestuário, couro e calçados',
            '2022160': 'Madeira, celulose e papel',
            '2022190': 'Produtos derivados do petróleo e biocombustíveis',
            '2022200': 'Indústria química, exceto petroquímica',
            '2022210': 'Indústria farmacêutica',
            '2022220': 'Artigos de borracha e de material plástico',
            '2022240': 'Metalurgia, siderurgia e produtos de minerais não-metálicos',
            '2022270': 'Máquinas, equipamentos e aparelhos diversos',
            '2022290': 'Veículos automotores e e outros equipamentos de transporte',
            '2022310': 'Móveis'
        }

    def get_7digit_sector(self, code: str) -> str:
        """Agrega código IPA para o nível de 7 dígitos"""
        if not code:
            return None
            
        code_str = str(code)
        
        # Primeiro verifica o mapeamento explícito
        if code_str in self.explicit_mapping:
            return self.explicit_mapping[code_str]
            
        # Se não estiver no mapeamento explícito, trunca para 7 dígitos
        if len(code_str) >= 7:
            return code_str[:7]
            
        return code_str

    def get_sector_name(self, code: str) -> str:
        """Obtém o nome do setor"""
        sector_code = self.get_7digit_sector(code)
        return self.sector_names.get(sector_code, f"Setor {sector_code}")

class DataLoader:
    """
    Classe responsável pelo carregamento e preparação de dados 
    com mapeamento setorial abrangente
    """
    
    def __init__(self, base_dir, ipa_path):
        """
        Inicializa o DataLoader com configurações de mapeamento

        Args:
            base_dir (str): Diretório base dos dados
            ipa_path (str): Caminho para o arquivo de índices de preços
        """
        self.base_dir = Path(base_dir)
        self.ipa_path = Path(ipa_path)
        self.sector_mapper = IPASectorMapper()
        
        # Definição centralizada dos padrões de contas
        self.padroes_contas = {
            # Contas fundamentais
            'variaveis_base': {
                'receita': [
                    'Receita liquida', 'Receita Líquida', 
                    'Receita líquida operacional'
                ],
                'cogs': [
                    'Custo Produtos', 'Custo dos Produtos', 
                    'CPV', 'Custo Produtos Vendidos'
                ],
                'trabalho_indireto': [
                    'Despesas admin', 'Despesas administrativ', 
                    'Despesas Administrativas',
                    'Despesas com Vendas', 'Despesas de Vendas', 
                    'Despesas comerciais'
                ],
                'imobilizado': [
                    'Imobilizado', 'IMOBILIZADO', 
                    'Ativo Imobilizado'
                ]
            }
        }
        
        # Definição das colunas de descrição possíveis
        self.desc_cols = [
            'Consolidado', 'Descrição', 'Conta', 
            'Item', 'Demonstrativo'
        ]
    
        # Mapeamentos setoriais
              
        self.excecoes = {
            'ABEV': '202211000', #ok
            'AGRO': '201101000', #ok
            'AGXY': '201101000', #ok
            'APTI': '202224200', #ok
            'AURA': '202107200', #ok
            'BAUH': '202210100', #ok
            'BDLL': '202228300', #ok
            'BEEF': '202210100', #ok
            'BRFS': '202210100', #ok
            'BRAV': '202106000', #ok
            'BRKM': '202219200', #ok
            'CAML': '202210900', #ok
            'CBAV': '202224400', #ok
            'CMIN': '202107000', #ok
            'CRPG': '202220100', #ok
            'CSNA': '202224200', #ok
            'CTCA': '201101000', #ok
            'DXCO': '202216000', #ok
            'EALT': '202224200', #ok
            'EMBR': '202230100', #ok
            'ETER': '202223300', #ok
            'EUCA': '202216000', #ok
            'FESA': '202224200', #ok
            'FHER': '202220100', #ok
            'FRAS': '202229400', #ok
            'FRIO': '202228200', #ok
            'GGBR': '202224200', #ok
            'GOAU': '202224200', #ok
            'GRAO': '201101000', #ok
            'HAGA': '202224200', #ok
            'INEP': '202227900', #ok
            'JALL': '202210700', #ok
            'JBSS': '202210100', #ok
            'JOPA': '202210900', #ok
            'KEPL': '202225900', #ok
            'KLBN': '202217900', #ok
            'LAND': '201101000', #ok
            'LUPA': '202228200', #ok
            'MDIA': '202210900', #ok
            'MGEL': '202224200', #ok
            'MNPR': '202210100', #ok
            'MRFG': '202210100', #ok
            'MSPA': '202217900', #ok
            'MTSA': '202228300', #ok
            'MWET': '202229400', #ok
            'NEMO': '202217100', #ok
            'NORD': '202224200', #ok
            'NUTR': '202220500', #ok
            'ODER': '202210300', #ok
            'OPCT': '202219900', #ok
            'OSXB': '202106000', #ok
            'PATI': '202106000', #ok
            'PETR': '202106000', #ok
            'PMAM': '202224200', #ok
            'POMO': '202229300', #ok
            'PRIO': '202106000', #ok
            'PTBL': '202223400', #ok
            'PTCA': '202227500', #ok
            'RAIZ': '202219300', #ok
            'RANI': '202217300', #ok
            'RAPT': '202229300', #ok
            'RCSL': '202229300', #ok
            'RECV': '202106000', #ok
            'ROMI': '202228400', #ok
            'RPMG': '202219200', #ok
            'RSUL': '202229400', #ok
            'SHUL': '202228100', #ok
            'SLCE': '201101000', #ok
            'SMTO': '202210700', #OK
            'SOJA': '201101000', #ok
            'SUZB': '202217100', #ok
            'TASA': '202225900', #ok
            'TKNO': '202225300', #ok
            'TTEN': '201101000', #ok
            'TUPY': '202229400', #ok
            'UNIP': '202220100', #ok
            'USIM': '202224200', #ok
            'VALE': '202107000', #ok
            'VITT': '202220500', #ok
            'INTB': '202226200', #ok
            'MLAS': '202226200', #ok
            'POSI': '202226200', #ok
            'ALPA': '202215000', #ok
            'CAMB': '202215000', #ok
            'CATA': '202213000', #ok
            'CEDO': '202213000', #ok
            'CTKA': '202213000', #ok
            'CTNM': '202213000', #ok
            'CTSA': '202213000', #ok
            'DOHL': '202213000', #ok
            'GRND': '202214000', #ok
            'MNDL': '202214000', #ok
            'PTNT': '202213000', #ok
            'SGPS': '202213000', #ok
            'TECN': '202214000', #ok
            'TEKA': '202213000', #ok
            'TFCO': '202214000', #ok
            'TXRX': '202213000', #ok
            'VIVA': '202214000', #ok
            'VULC': '202215000', #ok
            'WEGE': '202228100', #ok
            'BALM': '202228600', #ok
            'LMED': '202228600', #ok
            'BIOM': '202221200', #ok
            'NRTQ': '202221200', #ok 
            'OFSA': '202221200', #ok
            'WHRL': '202226200' #ok
            
        }
        
        self.pesos_ipa = {
            '201101000': 0.181212,  # Agriculta
            '202107000': 0.131045,  # Minerais metálicos ferrosos e não-ferrosos
            '202107200': 0.010652,  # Minerais metálicos não-ferrosos
            '202108000': 0.010956,  # Minerais não-metálicos
            '202106000': 0.058258,  # Petróleo e gás natural
            '202210100': 0.050661,  # Produtos de carne e derivados
            '202210200': 0.000774,  # Pescados
            '202210300': 0.004601,  # Conservas de frutas, legumes e outros vegetais
            '202210400': 0.028220,  # Óleos e gorduras vegetais
            '202210500': 0.012823,  # Laticínios
            '202210600': 0.013287,  # Produtos amiláceos e alimentos para animais
            '202210700': 0.012911,  # Açúcar
            '202210800': 0.003840,  # Café
            '202210900': 0.013359,  # Outros produtos alimentícios
            '202211000': 0.015163,  # Bebidas
            '202213000': 0.001005,  # Produtos Têxteis (fios, tecitos e artefatos têxteis)
            '202214000': 0.005509,  # Artigos do vestuário
            '202215000': 0.004814,  # Couro e calçados
            '202216000': 0.004704,  # Madeira desdobrada e produtos de madeira
            '202217100': 0.006317,  # Celulose
            '202217200': 0.003306,  # Papel, cartolina e papel-cartão
            '202217300': 0.003717,  # Embalagens de papel, papelão ondulado, cartolina e papel-cartão
            '202217900': 0.013340,  # Indústria de celulose e produtos de papel e papelão
            '202217400': 0.002853,  # Artefatos diversos de papel, cartolina e papel-cartão
            '202219200': 0.401483,  # Produtos derivados do petróleo
            '202219300': 0.015666,  # Biocombustíveis
            '202219900': 0.057149,  # Derivados do petróleo e Biocombustíveis
            '202220100': 0.016433,  # Produtos químicos inorgânicos
            '202220200': 0.006169,  # Produtos químicos orgânicos
            '202220300': 0.012223,  # Resinas e elastômeros
            '202220400': 0.000784,  # Fibras e filamentos sintéticos
            '202220500': 0.007952,  # Defensivos agrícolas e desinfestantes domissanitários
            '202220600': 0.007895,  # Produtos de limpeza, cosméticos e artigos de perfumaria e de higiene pessoal
            '202220700': 0.002970,  # Tintas, vernizes, esmaltes, lacas e produtos afins
            '202220900': 0.004247,  # Produtos e preparados químicos diversos
            '202221200': 0.010910,  # Medicamentos
            '202222100': 0.005115,  # Artigos de borracha
            '202222200': 0.013499,  # Produtos de material plástico
            '202223100': 0.002286,  # Vidro e produtos de vidro
            '202223200': 0.002511,  # Cimento
            '202223300': 0.003461,  # Artefatos de concreto, cimento e fibrocimento
            '202223400': 0.002983,  # Produtos cerâmicos
            '202223900': 0.003093,  # Pedras, cal e outros produtos não-metálicos
            '202224100': 0.003914,  # Ferro-gusa e ferro-ligas
            '202224200': 0.023399,  # Produtos siderúrgicos
            '202224400': 0.015491,  # Produtos da metalurgia dos não-ferrosos
            '202225300': 0.001792,  # Forjados e estampados
            '202225900': 0.008380,  # Produtos diversos de metal
            '202226200': 0.014526,  # Eletroeletrônicos e Eletrodomésticos
            '202227100': 0.005862,  # Geradores, transformadores e motores elétricos
            '202227200': 0.000732,  # Pilhas, baterias e acumuladores
            '202227300': 0.005389,  # Equipamentos para distribuição e controle de energia elétrica
            '202227900': 0.000517,  # Equipamentos e aparelhos elétricos diversos
            '202228100': 0.003944,  # Motores, bombas, compressores e equipamentos de transmissão
            '202228200': 0.006218,  # Máquinas e equipamentos de uso geral
            '202228300': 0.009082,  # Máquinas e equipamentos para a agricultura
            '202228400': 0.001261,  # Máquinas-ferramenta
            '202228500': 0.004068,  # Máquinas e equipamentos de terraplenagem e pavimentação
            '202228600': 0.002423,  # Máquinas e equipamentos de uso específico
            '202229100': 0.021207,  # Automóveis, camionetas e utilitários
            '202229200': 0.007699,  # Caminhões e ônibus
            '202229300': 0.002463,  # Cabines, carrocerias e reboques
            '202229400': 0.016004,  # Peças e acessórios para veículos automotores
            '202230100': 0.001045,  # Embarcações
            '202230300': 0.000437,  # Veículos ferroviários
            '202230900': 0.004383,  # Outros equipamentos de transporte
            '202231100': 0.004405  # Moveis 
        }

    def load_data(self):
        """
        Carrega, mapeia e prepara dados de todas as empresas
        
        Returns:
            DataFrame: Dados consolidados com mapeamento setorial
        """
        try:
            # 1. Carregar dados do IPA
            df_ipa = self._load_ipa_data()
            
            # 2. Identificar empresas nos diretórios
            companies = self._scan_directory()
            print(f"\nEmpresas identificadas: {len(companies)}")

            # 3. Processamento de dados
            dfs = []
            for company in companies:
                try:
                    # Processar dados cadastrais
                    cadastro = self._load_cadastral_data(company['cadastro'])
                    if cadastro is None:
                        continue

                    # Processar dados financeiros
                    df_fin = self._load_financial_data(company['financeiro'])
                    if df_fin is None:
                        continue
                    
                    # Consolidar e mapear dados
                    df_empresa = self._consolidate_and_map_company_data(cadastro, df_fin, df_ipa)
                    
                    if df_empresa is not None:
                        dfs.append(df_empresa)
                        print(f"Empresa {company['nome']} processada com sucesso")
                    
                except Exception as e:
                    print(f"Erro ao processar {company['nome']}: {e}")
                    continue

            # 4. Consolidar dados de todas as empresas
            if not dfs:
                raise ValueError("Nenhum dado válido carregado")

            df_raw = pd.concat(dfs, ignore_index=True)
            
            # 5. Limpeza final dos dados
            df_raw = self._clean_data(df_raw)
            
            return df_raw

        except Exception as e:
            print(f"Erro no carregamento: {str(e)}")
            return None

    def _scan_directory(self):
        """
        Escaneia diretórios para encontrar empresas
        
        Returns:
            list: Lista de empresas com arquivos cadastrais e financeiros
        """
        companies = []
        
        cadastro_dir = self.base_dir / 'cadastro'
        financeiro_dir = self.base_dir / 'financeiro'
        
        for pdf_file in cadastro_dir.glob('*.pdf'):
            nome = pdf_file.stem
            xlsx_file = financeiro_dir / f"{nome}.xlsx"
            
            if xlsx_file.exists():
                companies.append({
                    'nome': nome,
                    'cadastro': pdf_file,
                    'financeiro': xlsx_file
                })
                
        return companies

    def _load_cadastral_data(self, pdf_file):
        """
        Carrega dados cadastrais do PDF
        
        Args:
            pdf_file (Path): Caminho do arquivo PDF
        
        Returns:
            dict: Dados cadastrais da empresa
        """
        try:
            doc = fitz.open(pdf_file)
            texto = " ".join(pagina.get_text() for pagina in doc)
            texto_limpo = ' '.join(texto.split())

            def extrair_codigo_empresa(texto_limpo):
                try:
                    inicio = texto_limpo.find('Código')
                    if inicio != -1:
                        inicio += len('Código')
                        fim = texto_limpo.find('Bolsa/Fonte', inicio)
                        if fim != -1:
                            codigo = texto_limpo[inicio:fim].strip().split()[0]
                            if len(codigo) >= 4:
                                return codigo
                except:
                    pass

                try:
                    nome_arquivo = str(pdf_file.stem)
                    codigo = nome_arquivo.split('_')[0].strip()
                    if len(codigo) >= 4:
                        return codigo
                except:
                    pass

                return None

            def extrair_cnpj(texto_limpo):
                import re
                matches = re.findall(r'CNPJ[^0-9]*([0-9]{14})', texto_limpo)
                if matches:
                    cnpj = matches[0]
                    return f"{cnpj[:2]}.{cnpj[2:5]}.{cnpj[5:8]}/{cnpj[8:12]}-{cnpj[12:]}"
                return None

            def extrair_naics(texto_limpo):
                try:
                    inicio = texto_limpo.lower().find('setor naics nível 3')
                    if inicio != -1:
                        inicio += len('setor naics nível 3')
                        fim = texto_limpo.lower().find('setor', inicio)
                        if fim != -1:
                            naics = texto_limpo[inicio:fim].strip()
                            return naics.lower()
                except:
                    pass
                return None

            def extrair_subsetor_bovespa(texto_limpo):
                try:
                    texto_lower = texto_limpo.lower()
                    for padrao in [
                        'subsetor bovespa', 
                        'segmento bovespa', 
                        'subsetor', 
                        'segmento'
                    ]:
                        pos = texto_lower.find(padrao)
                        if pos != -1:
                            subsetor = texto_lower[pos+len(padrao):].strip().split()[0]
                            return subsetor
                except:
                    pass
                return None

            # Extrair informações
            codigo_empresa = extrair_codigo_empresa(texto_limpo)
            if codigo_empresa is None:
                print(f"Erro: Não foi possível extrair o código da empresa do arquivo {pdf_file}")
                return None

            cnpj = extrair_cnpj(texto_limpo)
            setor_naics = extrair_naics(texto_limpo)
            subsetor_bovespa = extrair_subsetor_bovespa(texto_limpo)

            cadastro = {
                'empresa': codigo_empresa,
                'cnpj': cnpj,
                'setor_naics3': setor_naics or 'não classificado',
                'subsetor_bovespa': subsetor_bovespa or 'não classificado'
            }

            print(f"\nDados extraídos para {cadastro['empresa']}:")
            print(f"CNPJ: {cadastro['cnpj']}")
            print(f"Setor NAICS3: {cadastro['setor_naics3']}")
            print(f"Subsetor Bovespa: {cadastro['subsetor_bovespa']}")

            return cadastro

        except Exception as e:
            print(f"Erro ao processar arquivo PDF {pdf_file}: {str(e)}")
            return None

    def _load_financial_data(self, xlsx_file):
        """
        Carrega dados financeiros para o modelo de dois insumos (um variável e um fixo).

        No modelo de dois insumos:
        - Insumo Variável (variable_input) = COGS + Trabalho Indireto
        - Insumo Fixo (capital) = Imobilizado + Investimentos + Intangíveis

        O método:
        1. Identifica e extrai os componentes básicos (COGS e trabalho indireto)
        2. Calcula o insumo variável composto
        3. Obtém o Insumo Fixo (capital) a partir de seus componentes
        4. Organiza os dados em uma estrutura temporal consistente

        Args:
            xlsx_file (Path): Caminho do arquivo Excel

        Returns:
            DataFrame: Dados financeiros processados com a estrutura do modelo de dois insumos
        """
        try:
            # 1. Carregar arquivo Excel
            df = pd.read_excel(xlsx_file)

            # 2. Identificar coluna de descrição
            desc_cols = ['Consolidado', 'Descrição', 'Conta', 'Item', 'Demonstrativo']
            desc_col = next((col for col in desc_cols if col in df.columns), df.columns[0])

            # 3. Extrair componentes usando padrões da classe
            dados_base = {}

            # Processar cada componente necessário
            for var, padroes in self.padroes_contas['variaveis_base'].items():
                for padrao in padroes:
                    mask = df[desc_col].str.contains(padrao, na=False, case=False)
                    if mask.any():
                        idx = mask.idxmax()
                        dados_base[var] = pd.to_numeric(df.iloc[idx, 1:].values, errors='coerce')
                        print(f"Encontrado {var} com padrão: {padrao}")
                        break

            """
            # 4.1 Verificar componentes essenciais_variable_input
            componentes_essenciais_variable_input = ['cogs', 'trabalho_indireto']
            if not all(comp in dados_base for comp in componentes_essenciais_variable_input):
                missing = [comp for comp in componentes_essenciais_variable_input if comp not in dados_base]
                print(f"Aviso: Componentes essenciais de variable_input faltando: {', '.join(missing)}")
                return None

            # 4.2 Verificar componentes essenciais_capital
            componentes_essenciais_capital = ['imobilizado', 'investimentos', 'intangiveis']
            if not all(comp in dados_base for comp in componentes_essenciais_capital):
                missing = [comp for comp in componentes_essenciais_capital if comp not in dados_base]
                print(f"Aviso: Componentes essenciais de capital faltando: {', '.join(missing)}")
                return None            
            
           """ 
            
            # 4.1 Verificar componentes essenciais_variable_input
            componentes_essenciais_variable_input = ['cogs', 'trabalho_indireto']
            if not all(comp in dados_base for comp in componentes_essenciais_variable_input):
                missing = [comp for comp in componentes_essenciais_variable_input if comp not in dados_base]
                print(f"Aviso: Componentes essenciais de variable_input faltando: {', '.join(missing)}")
                return None

            # 4.2 Verificar componentes do capital
            componentes_capital = ['imobilizado']
            componentes_capital_disponiveis = [comp for comp in componentes_capital if comp in dados_base]

            if not componentes_capital_disponiveis:
                print(f"Aviso: Nenhum componente de capital encontrado entre: {', '.join(componentes_capital)}")
                return None

            # 5. Construir DataFrame com a estrutura do modelo de dois insumos
            datas = pd.to_datetime(df.columns[1:])

            # Calcular insumo variável composto (COGS + Trabalho Indireto)
            variable_input = dados_base['cogs'] + dados_base['trabalho_indireto']

            # Calcular capital ampliado (usando apenas componentes disponíveis)
            capital = pd.Series(0, index=datas)
            for comp in componentes_capital_disponiveis:
                capital += dados_base[comp]

            # Imprimir estatísticas sobre os componentes de capital
            print(f"Composição do capital:")
            for comp in componentes_capital_disponiveis:
                print(f"- {comp.capitalize()} médio: {dados_base[comp].mean():.2f}")
            print(f"- Capital total médio: {capital.mean():.2f}")

            # Criar DataFrame final com a estrutura do modelo
            dados_modelo = {
                'variable_input': variable_input,      # Insumo variável composto
                'capital': capital,                    # Capital (imobilizado)
                'receita': dados_base.get('receita'),  # Receita para cálculos de share
            }

            df_final = pd.DataFrame(dados_modelo, index=datas).T

            # 6. Validação final
            if df_final.empty:
                print("Aviso: DataFrame final está vazio")
                return None

            # Verificar validade dos dados
            n_valid = (~df_final.isnull()).sum().min()
            print(f"\nEstatísticas do carregamento:")
            print(f"Períodos: {len(df_final.columns)}")
            print(f"Observações válidas: {n_valid}")
            print(f"Composição do insumo variável:")
            print(f"- COGS médio: {dados_base['cogs'].mean():.2f}")
            print(f"- Trabalho indireto médio: {dados_base['trabalho_indireto'].mean():.2f}")
            print(f"- Insumo variável médio: {variable_input.mean():.2f}")

            return df_final

        except Exception as e:
            print(f"Erro no carregamento dos dados financeiros: {str(e)}")
            return None

    def _consolidate_and_map_company_data(self, cadastro, df_fin, df_ipa):
        """
        Consolida dados de uma empresa com mapeamento setorial completo
        
        Args:
            cadastro (dict): Dados cadastrais da empresa
            df_fin (DataFrame): Dados financeiros
            df_ipa (DataFrame): Dados do índice de preços
        
        Returns:
            DataFrame: Dados consolidados e mapeados
        """
        try:
            if cadastro is None or df_fin is None:
                return None

            registros = []
            for data in df_fin.columns:
                # Registro base
                registro = {
                    'empresa': cadastro['empresa'],
                    'cnpj': cadastro['cnpj'],
                    'setor_naics3': cadastro['setor_naics3'],
                    'subsetor_bovespa': cadastro['subsetor_bovespa'],
                    'data': data,
                    'ano': data.year,
                    'trimestre': (data.month - 1) // 3 + 1
                }

                # Adicionar variáveis financeiras
                for var in df_fin.index:
                    registro[var] = df_fin.loc[var, data]

                # Mapear setor para IPA
                registro_serie = pd.Series(registro)
                setor_ipa, fonte_mapeamento, peso_ipa = self._map_company_to_ipa(registro_serie)
                
                # Adicionar informações de mapeamento
                registro['setor_ipa'] = setor_ipa
                registro['fonte_mapeamento'] = fonte_mapeamento
                registro['peso_ipa'] = peso_ipa

                registros.append(registro)

            # Criar DataFrame
            df_empresa = pd.DataFrame(registros)

            return df_empresa

        except Exception as e:
            print(f"Erro na consolidação e mapeamento: {str(e)}")
            return None

    def _map_company_to_ipa(self, row):
        """
        Mapeia empresa para código IPA
        
        Args:
            row (Series): Linha de dados da empresa
        
        Returns:
            tuple: (código IPA, fonte de mapeamento, peso IPA)
        """
        empresa = row['empresa'].split()[0].upper()
        setor_naics = str(row.get('setor_naics3', '')).lower().strip()
        subsetor_bovespa = str(row.get('subsetor_bovespa', '')).lower().strip()

        # 1. Verificar exceções específicas
        if empresa in self.excecoes:
            codigo_ipa = self.excecoes[empresa]
            return codigo_ipa, 'exceção', self.pesos_ipa.get(codigo_ipa, 0)

        # 2. Mapear por NAICS
        for naics_key, codigo_ipa in self.naics_para_ipa.items():
            if naics_key in setor_naics:
                return codigo_ipa, 'naics', self.pesos_ipa.get(codigo_ipa, 0)

        # 3. Mapear por Bovespa
        for bovespa_key, codigo_ipa in self.bovespa_para_ipa.items():
            if bovespa_key in subsetor_bovespa:
                return codigo_ipa, 'bovespa', self.pesos_ipa.get(codigo_ipa, 0)

        # 4. Fallback padrão
        default_ipa = '202220200'  # Químicos orgânicos
        print(f"Aviso: Sem mapeamento para {empresa} (NAICS: {setor_naics}, Bovespa: {subsetor_bovespa})")
        return default_ipa, 'default', self.pesos_ipa.get(default_ipa, 0)

    def _load_ipa_data(self):
        """
        Carrega dados do índice IPA, tratando adequadamente números e datas em formato brasileiro.

        O método processa:
        - Números com separador de milhar como ponto (ex: '1.020,50')
        - Datas em múltiplos formatos brasileiros (ex: 'jan/22', 'jan-22', '01/22')

        Returns:
            DataFrame: Dados do IPA com índices e deflatores calculados
        """
        try:
            # Carrega o CSV com tratamento para números brasileiros
            df_ipa = pd.read_csv(
                self.ipa_path,
                sep=';',
                encoding='utf-8',
                decimal=',',
                thousands='.'
            )

            # Mapeamento de meses em português para inglês
            month_map = {
                'jan': 'Jan', 'fev': 'Feb', 'mar': 'Mar', 'abr': 'Apr',
                'mai': 'May', 'jun': 'Jun', 'jul': 'Jul', 'ago': 'Aug',
                'set': 'Sep', 'out': 'Oct', 'nov': 'Nov', 'dez': 'Dec'
            }

            # Normaliza meses e formato da data
            df_ipa['data'] = df_ipa['data'].str.lower()
            df_ipa['data'] = df_ipa['data'].replace(month_map, regex=True)

            def parse_date(date_str):
                """
                Tenta converter uma string de data usando múltiplos formatos.

                Args:
                    date_str: String contendo a data

                Returns:
                    datetime: Data convertida ou None se falhar
                """
                try:
                    # Tenta múltiplos formatos de data
                    for fmt in ['%b/%y', '%b-%y', '%m/%y', '%Y-%m']:
                        try:
                            return pd.to_datetime(date_str, format=fmt)
                        except ValueError:
                            continue

                    # Se nenhum formato funcionar, registra o problema
                    print(f"Formato de data não reconhecido: {date_str}")
                    return None

                except Exception as e:
                    print(f"Erro ao processar data {date_str}: {e}")
                    return None

            # Aplica o parsing de data com tratamento de erros
            df_ipa['data'] = df_ipa['data'].apply(parse_date)

            # Remove linhas com datas inválidas
            df_ipa = df_ipa.dropna(subset=['data'])

            # Define o índice e calcula o deflator
            df_ipa.set_index('data', inplace=True)
            base_value = df_ipa['indice'].iloc[-1]
            df_ipa['deflator'] = base_value / df_ipa['indice']

            return df_ipa

        except Exception as e:
            print(f"Erro no carregamento do IPA: {str(e)}")
            return None

    def _clean_data(self, df):
        """
        Cleans and standardizes raw financial data, focusing on basic data quality 
        without performing any markup-related calculations.

        This method handles:
        1. Basic data structure cleaning
        2. Variable type conversion and validation
        3. Treatment of missing and invalid values

        Markup-specific calculations (like input shares) are handled separately
        by the DLWMarkupEstimator class.

        Args:
            df (DataFrame): Raw financial data

        Returns:
            DataFrame: Cleaned and validated data
        """
        try:
            # 1. Basic Structure Cleaning
            df = df.drop_duplicates(subset=['empresa', 'data'])
            df = df.sort_values(['empresa', 'data'])

            # 2. Essential Variables - Required for later markup estimation
            essential_vars = {
                'receita': True,           # Revenue 
                'cogs': True,              # Cost of goods sold
                'trabalho_indireto': True, # Indirect labor costs
                'capital': True            # Fixed capital (renamed from imobilizado)
            }

            # 3. Supporting Variables
            supporting_vars = {
                'ativo_total': True,      # Balance sheet total
                'data': False,            # Time identifier
                'empresa': False,         # Firm identifier
                'setor_ipa': False,       # Sector identifier
                'peso_ipa': False         # Price index weight
            }

            # 4. Convert and Validate Variables
            for var, non_negative in {**essential_vars, **supporting_vars}.items():
                if var in df.columns:
                    # Skip non-numeric variables
                    if var in ['data', 'empresa', 'setor_ipa']:
                        continue

                    # Convert to numeric
                    df[var] = pd.to_numeric(df[var], errors='coerce')

                    # Apply non-negative constraint where applicable
                    if non_negative:
                        invalid_mask = df[var] <= 0
                        if invalid_mask.any():
                            print(f"Removing {invalid_mask.sum()} non-positive values from {var}")
                            df.loc[invalid_mask, var] = np.nan

            # 5. Remove observations missing essential variables
            essential_columns = list(essential_vars.keys())
            df = df.dropna(subset=essential_columns)

            # 6. Get variables using existing method
            df['variable_input'] = self._calculate_variable_input_costs(df)
            df['capital'] = self._calculate_capital_costs(df)

            # 7. Final Quality Check
            clean_obs = len(df)
            if clean_obs == 0:
                raise ValueError("No valid observations after cleaning")

            print(f"Final clean dataset contains {clean_obs} observations")

            return df

        except Exception as e:
            print(f"Error in data cleaning: {str(e)}")
            return df

    def aggregate_sectors(self, df_deflated):
        """
        Agrega setores após deflação

        Args:
            df_deflated (DataFrame): DataFrame com dados já deflacionados

        Returns:
            DataFrame: Dados com setores agregados
        """
        try:
            logging.info("Iniciando agregação setorial...")
            df_final = df_deflated.copy()
            sector_mapper = IPASectorMapper()

            # Adicionar classificação setorial agregada
            logging.info("Mapeando códigos setoriais...")
            df_final['setor_ipa_7d'] = df_final['setor_ipa'].apply(sector_mapper.get_7digit_sector)
            df_final['nome_setor'] = df_final['setor_ipa_7d'].apply(sector_mapper.get_sector_name)

            # Validar mapeamento
            unmapped = df_final[df_final['setor_ipa_7d'].isna()]
            if len(unmapped) > 0:
                logging.warning(f"{len(unmapped)} observações sem mapeamento setorial")
                logging.warning(f"Códigos não mapeados: {unmapped['setor_ipa'].unique()}")

            # Log de resultados da agregação
            n_original = df_final['setor_ipa'].nunique()
            n_aggregated = df_final['setor_ipa_7d'].nunique()
            logging.info(f"Setores originais: {n_original}")
            logging.info(f"Setores após agregação: {n_aggregated}")

            # Validar consistência do mapeamento
            mapping_check = df_final.groupby(['setor_ipa', 'setor_ipa_7d', 'nome_setor']).size().reset_index()
            logging.info("\nMapeamento de setores:")
            for _, row in mapping_check.iterrows():
                logging.info(f"{row['setor_ipa']} -> {row['setor_ipa_7d']} ({row['nome_setor']})")

            return df_final

        except Exception as e:
            logging.error(f"Erro na agregação setorial: {str(e)}")
            traceback.print_exc()
            return df_deflated

        
class IPADeflator:
    def apply_weighted_deflator(self, df, df_ipa):
        """
        Applies weighted deflation to monetary variables.
        """
        try:
            # Get base period index value
            indice_base = df_ipa['indice'].iloc[-1]

            # Filter data to periods with available price indices
            data_minima_ipa = df_ipa.index.min()
            df_deflated = df[df['data'] >= data_minima_ipa].copy()

            # Convert index to period for proper matching
            df_ipa.index = df_ipa.index.to_period('M')

            # Variables to deflate
            monetary_vars = ['variable_input', 'capital', 'imobilizado', 'receita', 'cogs', 'trabalho_indireto']

            # Apply deflation to each variable
            for var in monetary_vars:
                if var not in df_deflated.columns:
                    logging.warning(f"Variable {var} not found in dataset")
                    continue

                # Store nominal values
                df_deflated[f'{var}_nominal'] = df_deflated[var]

                # Apply deflation only to non-null values
                for idx, row in df_deflated.iterrows():
                    if pd.isnull(row[var]):
                        continue

                    periodo = row['data'].to_period('M')
                    if periodo in df_ipa.index:
                        indice_periodo = df_ipa.loc[periodo, 'indice']
                        deflator = (indice_base / indice_periodo) ** row['peso_ipa']

                        # Safe multiplication with null check
                        current_value = df_deflated.loc[idx, var]
                        if pd.notnull(current_value):
                            df_deflated.loc[idx, var] = current_value * deflator

            # Validate results
            for var in monetary_vars:
                if var in df_deflated.columns:
                    null_count = df_deflated[var].isnull().sum()
                    if null_count > 0:
                        logging.warning(f"{null_count} null values in deflated {var}")

            return df_deflated

        except Exception as e:
            logging.error(f"Error in deflation process: {str(e)}")
            raise

def print_sector_summary(df, sector_mapper):
    """
    Imprime uma tabela síntese com estatísticas descritivas por setor.

    Args:
        df (pd.DataFrame): DataFrame com os dados finais processados.
        sector_mapper (IPASectorMapper): Instância da classe IPASectorMapper para mapeamento de setores.
    """
    # Verificar se a coluna de setor agregado existe
    if 'setor_ipa_7d' not in df.columns:
        logging.error("Coluna 'setor_ipa_7d' não encontrada no DataFrame.")
        return

    # Obter contagens, médias e desvios padrão por setor
    sector_summary = df.groupby('setor_ipa_7d').agg(
        empresas=('empresa', 'nunique'),
        observacoes=('empresa', 'count'),
        receita_media=('receita', 'mean'),
        receita_desvio=('receita', 'std'),
        input_var_medio=('variable_input', 'mean'),
        input_var_desvio=('variable_input', 'std'),
        capital_medio=('capital', 'mean'),
        capital_desvio=('capital', 'std')
    ).reset_index()

    # Renomear coluna de setor para o nome
    sector_summary['setor'] = sector_summary['setor_ipa_7d'].map(lambda x: sector_mapper.get_sector_name(x))

    # Reordenar colunas
    sector_summary = sector_summary[['setor', 'empresas', 'observacoes', 
                                     'receita_media', 'receita_desvio',
                                     'input_var_medio', 'input_var_desvio',
                                     'capital_medio', 'capital_desvio']]

    # Formatar valores numéricos
    numeric_cols = ['receita_media', 'receita_desvio',
                    'input_var_medio', 'input_var_desvio',
                    'capital_medio', 'capital_desvio']
    sector_summary[numeric_cols] = sector_summary[numeric_cols].applymap(lambda x: f"{x:,.2f}")

    # Imprimir tabela
    print("\nResumo por Setor:")
    print(sector_summary.to_string(index=False))

if __name__ == '__main__':
    try:
        # Configure directories and logging
        base_dir = Path(r'C:\Users\fabio\OneDrive\Projeto_Markup\Data\raw')
        ipa_path = Path(r'C:\Users\fabio\OneDrive\Projeto_Markup\Data\Index\ipa_data.csv')
        output_dir = Path(r'C:\Users\fabio\OneDrive\Projeto_Markup\Results\Model3\v1')
        
        # Create output directory if it doesn't exist
        output_dir.mkdir(parents=True, exist_ok=True)
        
        # Set up logging configuration
        log_path = output_dir / 'estimation_log.txt'
        logging.basicConfig(
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s',
            handlers=[
                logging.FileHandler(log_path),
                logging.StreamHandler()
            ]
        )
        
        logging.info("Starting two-input markup estimation process")
        
        # Validate input files
        if not base_dir.exists():
            raise FileNotFoundError(f"Base directory not found: {base_dir}")
        if not ipa_path.exists():
            raise FileNotFoundError(f"IPA file not found: {ipa_path}")
            
        # Initialize our main classes
        data_loader = DataLoader(base_dir, ipa_path)
        deflator = IPADeflator()
        
        # Step 1: Load and clean raw data
        logging.info("Loading and cleaning raw data...")
        df_raw = data_loader.load_data()
        
        if df_raw is None or df_raw.empty:
            raise ValueError("Failed to load valid data")
            
        if df_raw is not None:
            # Check data quality before proceeding
            null_counts = df_raw.isnull().sum()
            logging.info("\nMissing value summary:")
            for col, count in null_counts.items():
                if count > 0:
                    logging.info(f"{col}: {count} missing values ({count/len(df_raw)*100:.1f}%)")

            # Validate variable input calculation
            if 'variable_input' in df_raw.columns:
                invalid_inputs = df_raw['variable_input'].isnull().sum()
                logging.info(f"\nVariable input calculation:")
                logging.info(f"Invalid/missing values: {invalid_inputs}")
        
        
        # Log initial data statistics
        logging.info("Raw data loaded:")
        logging.info(f"Total observations: {len(df_raw)}")
        logging.info(f"Unique firms: {df_raw['empresa'].nunique()}")
        logging.info(f"Time period: {df_raw['data'].min():%Y-%m} to {df_raw['data'].max():%Y-%m}")
        
        # Step 2: Load price indices and apply deflation
        logging.info("Loading price indices and applying deflation...")
        df_ipa = data_loader._load_ipa_data()
        df_deflated = deflator.apply_weighted_deflator(df_raw, df_ipa)
        
        # Step 3: Aggregate sectors
        logging.info("Performing sector aggregation...")
        df_final = data_loader.aggregate_sectors(df_deflated)
        
        # Validação da agregação - ADICIONAR ESTA VERIFICAÇÃO
        if 'setor_ipa_7d' not in df_final.columns:
            raise ValueError("Sector aggregation failed: setor_ipa_7d column not created")
        
        # Save processed datasets
        logging.info("Saving processed datasets...")
        logging.info("\nSetor aggregation results:")
        logging.info(f"Original sectors (setor_ipa): {df_final['setor_ipa'].nunique()}")
        logging.info(f"Aggregated sectors (setor_ipa_7d): {df_final['setor_ipa_7d'].nunique()}")
        
        # Save with clear naming convention
        df_raw.to_excel(output_dir / 'raw_data.xlsx', index=False)
        df_deflated.to_excel(output_dir / 'deflated_data.xlsx', index=False)
        df_final.to_excel(output_dir / 'final_data.xlsx', index=False)
        
        # Criar instância do IPASectorMapper
        sector_mapper = IPASectorMapper()
        
        # Imprimir resumo por setor
        print_sector_summary(df_final, sector_mapper)
        
        # Log final summary statistics
        logging.info("\nFinal Dataset Summary:")
        logging.info(f"Total observations: {len(df_final)}")
        logging.info(f"Unique firms: {df_final['empresa'].nunique()}")
        logging.info(f"Original IPA sectors: {df_final['setor_ipa'].nunique()}")
        logging.info(f"Aggregated sectors: {df_final['setor_ipa_7d'].nunique()}")
        
        # Calculate and log data quality metrics
        missing_rates = df_final.isnull().mean() * 100
        logging.info("\nData Quality Metrics:")
        for col, rate in missing_rates.items():
            if rate > 0:
                logging.info(f"Missing rate for {col}: {rate:.1f}%")
                
        logging.info("Processing completed successfully!")
        print(f"\nResults saved in: {output_dir}")
        
    except Exception as e:
        logging.error(f"Fatal error in processing: {str(e)}")
        traceback.print_exc()
        raise

2025-03-05 16:35:11,219 - INFO - Starting two-input markup estimation process
2025-03-05 16:35:11,226 - INFO - Loading and cleaning raw data...



Empresas identificadas: 127

Dados extraídos para ABEV:
CNPJ: None
Setor NAICS3: indústria de bebidas
Subsetor Bovespa: não classificado
Encontrado receita com padrão: Receita liquida
Encontrado cogs com padrão: Custo Produtos
Encontrado trabalho_indireto com padrão: Despesas admin
Encontrado imobilizado com padrão: Imobilizado
Composição do capital:
- Imobilizado médio: 4328758.57
- Capital total médio: 4328758.57

Estatísticas do carregamento:
Períodos: 148
Observações válidas: 3
Composição do insumo variável:
- COGS médio: 3529056.91
- Trabalho indireto médio: 492973.64
- Insumo variável médio: 4022030.55
Empresa ABEV processada com sucesso

Dados extraídos para AGRO:
CNPJ: 07.628.528/0001-59
Setor NAICS3: -
Subsetor Bovespa: agropecuária
Encontrado receita com padrão: Receita liquida
Encontrado cogs com padrão: Custo Produtos
Encontrado trabalho_indireto com padrão: Despesas admin
Encontrado imobilizado com padrão: Imobilizado
Composição do capital:
- Imobilizado médio: 93755.38
-

2025-03-05 16:36:02,527 - INFO - 
Missing value summary:
2025-03-05 16:36:02,528 - INFO - cnpj: 382 missing values (3.6%)
2025-03-05 16:36:02,529 - INFO - variable_input: 70 missing values (0.7%)
2025-03-05 16:36:02,531 - INFO - capital: 441 missing values (4.1%)
2025-03-05 16:36:02,533 - INFO - receita: 1629 missing values (15.3%)
2025-03-05 16:36:02,534 - INFO - 
Variable input calculation:
2025-03-05 16:36:02,536 - INFO - Invalid/missing values: 70
2025-03-05 16:36:02,537 - INFO - Raw data loaded:
2025-03-05 16:36:02,537 - INFO - Total observations: 10651
2025-03-05 16:36:02,541 - INFO - Unique firms: 107
2025-03-05 16:36:02,543 - INFO - Time period: 1986-03 to 2024-12
2025-03-05 16:36:02,545 - INFO - Loading price indices and applying deflation...


Encontrado receita com padrão: Receita liquida
Encontrado cogs com padrão: Custo Produtos
Encontrado trabalho_indireto com padrão: Despesas admin
Encontrado imobilizado com padrão: Imobilizado
Composição do capital:
- Imobilizado médio: 621448.17
- Capital total médio: 621448.17

Estatísticas do carregamento:
Períodos: 123
Observações válidas: 3
Composição do insumo variável:
- COGS médio: 2190375.21
- Trabalho indireto médio: 121495.78
- Insumo variável médio: 2311870.98
Empresa WHRL processada com sucesso
Removing 1605 non-positive values from receita
Removing 427 non-positive values from capital
Error in data cleaning: ['cogs', 'trabalho_indireto']


2025-03-05 16:36:11,776 - WARNING - Variable imobilizado not found in dataset
2025-03-05 16:36:16,060 - WARNING - Variable cogs not found in dataset
2025-03-05 16:36:16,060 - WARNING - Variable trabalho_indireto not found in dataset
2025-03-05 16:36:16,061 - WARNING - 31 null values in deflated variable_input
2025-03-05 16:36:16,063 - WARNING - 430 null values in deflated capital
2025-03-05 16:36:16,066 - WARNING - 1600 null values in deflated receita
2025-03-05 16:36:16,070 - INFO - Performing sector aggregation...
2025-03-05 16:36:16,072 - INFO - Iniciando agregação setorial...
2025-03-05 16:36:16,074 - INFO - Mapeando códigos setoriais...
2025-03-05 16:36:16,091 - INFO - Setores originais: 39
2025-03-05 16:36:16,092 - INFO - Setores após agregação: 11
2025-03-05 16:36:16,099 - INFO - 
Mapeamento de setores:
2025-03-05 16:36:16,100 - INFO - 201101000 -> 2011010 (Agricultura e pecuária)
2025-03-05 16:36:16,103 - INFO - 202106000 -> 2021060 (Indústria extrativa, inclusive petróleo e gá


Resumo por Setor:
                                                      setor  empresas  observacoes receita_media receita_desvio input_var_medio input_var_desvio capital_medio capital_desvio
                                     Agricultura e pecuária         8          338    814,018.27   1,418,095.03      609,706.48     1,089,476.89    544,526.48   1,167,962.84
      Indústria extrativa, inclusive petróleo e gás natural         9          604 39,434,596.92  77,640,890.25   17,603,681.59    41,115,053.50 65,953,135.45 158,709,241.85
                                        Alimentos e bebidas        13         1167  4,723,543.68   8,122,231.16    2,984,515.04     6,189,882.86  2,579,916.27   3,951,854.86
                       Têxteis, vestuário, couro e calçados        18         1952    238,381.99     408,034.40      149,933.86       249,773.56    126,134.23     176,507.90
                                  Madeira, celulose e papel         7          815  1,920,329.85   4,035,764.58